In [2]:
%matplotlib widget
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import mplcursors
import numpy as np

# Load the data from 'WWI map clean.csv' using pandas
df = pd.read_csv('WWI_Bombing_Operations_Cleaned.csv')

# Load the GeoJSON file
gdf = gpd.read_file('europe-1914.geojson')

# Create a GeoDataFrame from the WWI data
wwi_gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df['Target Longitude'], df['Target Latitude']))

wwi_gdf['Year'] = pd.to_datetime(wwi_gdf['Mission Date']).dt.year

color_map = {
    'ITALY': 'green',
    'USA': 'blue',
    'UK': 'red',
}

# Function to update plot based on the selected year
def update_plot(year):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12))
    
    gdf.plot(ax=ax1, color='lightgray', edgecolor='black')
    
    # Filter the data for the selected year
    filtered_gdf = wwi_gdf[wwi_gdf['Year'] == year]
    
    # Plot each country's data in the filtered GeoDataFrame
    for country, color in color_map.items():
        country_gdf = filtered_gdf[filtered_gdf['Country of Service'] == country]
        if not country_gdf.empty:
            country_gdf.plot(ax=ax1, markersize=20, alpha=0.5, color=color, label=country)

    # Calculate the bounding box of the data points
    min_lon, min_lat, max_lon, max_lat = filtered_gdf.total_bounds
    
    # Create a buffer around the bounding box
    buffer = 1.0  # Adjust the buffer size as needed
    min_lon -= buffer
    min_lat -= buffer
    max_lon += buffer
    max_lat += buffer
    
    # Set the map extent to the bounding box of the data points
    ax1.set_xlim(min_lon, max_lon)
    ax1.set_ylim(min_lat, max_lat)
    
    ax1.set_title(f'WWI Bombings by Country of Service - Year {year}')
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.legend(title='Country of Service')
    
    # Plot the GeoDataFrame for the altitude-based heatmap
    gdf.plot(ax=ax2, color='lightgray', edgecolor='black')
    
    # Overlay the WWI data as a heatmap with colors based on altitude
    filtered_gdf.plot(ax=ax2, markersize=20, alpha=0.5, c='ALTITUDE', cmap='viridis', legend=True)

    # Set plot title and labels
    ax2.set_title(f'WWI Bombings by Altitude - Year {year}')
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Latitude')

    # Set the map extent to the bounding box of the data points
    ax2.set_xlim(min_lon, max_lon)
    ax2.set_ylim(min_lat, max_lat)

    # Create a bar graph showing the number of missions per aircraft for the selected year
    aircraft_counts = filtered_gdf['Aircraft'].value_counts()
    colors = plt.cm.Paired(np.linspace(0, 1, len(aircraft_counts)))
    ax3.bar(aircraft_counts.index, aircraft_counts.values, color=colors)
    ax3.set_title(f'Number of Missions per Aircraft - Year {year}')
    ax3.set_xlabel('Aircraft')
    ax3.set_ylabel('Number of Missions')
    ax3.tick_params(axis='x', rotation=45)

    # Create a bar graph showing the total bomb load dropped by each country for the selected year
    country_bombloads = filtered_gdf.groupby('Country of Service')['BOMBLOAD'].sum()
    ax4.bar(country_bombloads.index, country_bombloads.values)
    ax4.set_title(f'Total Bomb Load Dropped by Country - Year {year}')
    ax4.set_xlabel('Country')
    ax4.set_ylabel('Total Bomb Load (lbs)')
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()  # Adjust the layout for better spacing
    
    # Add hovering functionality to the points in the left side map
    cursor1 = mplcursors.cursor(ax1, hover=True)
    
    @cursor1.connect("add")
    def on_add_top_left(sel):
        # Get the x and y coordinates of the selected point
        x, y = sel.target
        
        # Find the rows in the filtered_gdf that match the selected point
        matching_rows = filtered_gdf[(filtered_gdf['Target Longitude'] == x) & (filtered_gdf['Target Latitude'] == y)]
        
        if not matching_rows.empty:
            # If matching rows are found, select the first row
            row = matching_rows.iloc[0]
            
            # Set the annotation text with the desired information
            sel.annotation.set_text(f"Mission Date: {row['Mission Date']}\nTarget Location: {row['TGTLOCATION']}\nAltitude: {row['ALTITUDE']}\nTarget Type: {row['Target Type']}")
        else:
            # If no matching rows are found, display a default message or handle it accordingly
            sel.annotation.set_text("No data available for the selected point.")
    
    # Add hovering functionality to the points in the right side map
    cursor2 = mplcursors.cursor(ax2, hover=True)
    
    @cursor2.connect("add")
    def on_add_top_right(sel):
        # Get the x and y coordinates of the selected point
        x, y = sel.target
        
        # Find the rows in the filtered_gdf that match the selected point
        matching_rows = filtered_gdf[(filtered_gdf['Target Longitude'] == x) & (filtered_gdf['Target Latitude'] == y)]
        
        if not matching_rows.empty:
            # If matching rows are found, select the first row
            row = matching_rows.iloc[0]
            
            # Set the annotation text with the desired information
            sel.annotation.set_text(f"Mission Date: {row['Mission Date']}\nTarget Location: {row['TGTLOCATION']}\nAltitude: {row['ALTITUDE']}\nTarget Type: {row['Target Type']}")
        else:
            # If no matching rows are found, display a default message or handle it accordingly
            sel.annotation.set_text("No data available for the selected point.")

    # Add hovering functionality to the bars in the bottom left chart
    cursor3 = mplcursors.cursor(ax3, hover=True)
    
    @cursor3.connect("add")
    def on_add_bottom_left(sel):
        # Get the index of the selected bar
        index = sel.target.index
        
        # Get the aircraft name and count
        aircraft = aircraft_counts.index[index]
        count = aircraft_counts.values[index]
        
        # Set the annotation text with the aircraft name and count
        sel.annotation.set_text(f"Aircraft: {aircraft}\nMissions: {count}")
    
    plt.show()

# Create a slider widget for years
year_slider = widgets.IntSlider(
    value=1918,
    min=wwi_gdf['Year'].min(),
    max=wwi_gdf['Year'].max(),
    step=1,
    description='Year:',
    continuous_update=False
)

# Display the widget
display(year_slider)

# Use 'interactive_output' to link the slider and the plotting function
out = widgets.interactive_output(update_plot, {'year': year_slider})

# Display the interactive output
display(out)

IntSlider(value=1918, continuous_update=False, description='Year:', max=1918, min=1915)

Output()